# Medicinal Plant Detection AI — Implementation Notebook

This notebook implements the **Phase 1–5** pipeline using the **Indian Medicinal Leaves** dataset from Kaggle. It is organized to be a practical demo for your project guide.

**Before running:**
1. Download the dataset from Kaggle (https://www.kaggle.com/datasets/aryashah2k/indian-medicinal-leaves-dataset) and
   extract it into `./dataset/` such that images are inside subfolders per class: `dataset/<class_name>/*.jpg`.
2. Use a Python 3.8+ environment. Recommended: create a virtualenv and install dependencies listed in the first cell.


In [ ]:
# Install required packages (uncomment and run if needed)
!pip install -q tensorflow==2.12.0 opencv-python-headless matplotlib scikit-learn pandas streamlit nbconvert
# If running on Colab, use: 
# !pip install -q tensorflow==2.12.0 opencv-python-headless

print('Install packages if needed and ensure dataset is placed at ./dataset/')


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import sqlite3

print('TensorFlow version:', tf.__version__)


## Phase 1 — Dataset Preparation

Load dataset directory structure and show class distribution.

In [ ]:
DATA_DIR = Path('dataset')
assert DATA_DIR.exists(), 'Please download and extract dataset to ./dataset'

# list classes
classes = [p.name for p in DATA_DIR.iterdir() if p.is_dir()]
print('Found classes:', len(classes))
for c in classes[:20]:
    print('-', c)

# quick count
counts = {c: len(list((DATA_DIR/c).glob('*'))) for c in classes}
counts_df = pd.DataFrame(list(counts.items()), columns=['class','count']).sort_values('count', ascending=False)
counts_df.head()


### Create train/val/test splits and data generators (ImageDataGenerator)

In [ ]:
# Create directory structure for train/val/test splits (if not already present)
from sklearn.model_selection import train_test_split

ROOT = Path('data_split')
if not ROOT.exists():
    ROOT.mkdir()
    for split in ['train','val','test']:
        (ROOT/split).mkdir(exist_ok=True)

    # split per class
    for cls in classes:
        imgs = list((DATA_DIR/cls).glob('*'))
        imgs = [p for p in imgs if p.suffix.lower() in ['.jpg','.jpeg','.png']]
        train, temp = train_test_split(imgs, test_size=0.3, random_state=42)
        val, test = train_test_split(temp, test_size=0.5, random_state=42)
        for p in train:
            (ROOT/'train'/cls).mkdir(parents=True, exist_ok=True)
            os.symlink(p.resolve(), (ROOT/'train'/cls/p.name))
        for p in val:
            (ROOT/'val'/cls).mkdir(parents=True, exist_ok=True)
            os.symlink(p.resolve(), (ROOT/'val'/cls/p.name))
        for p in test:
            (ROOT/'test'/cls).mkdir(parents=True, exist_ok=True)
            os.symlink(p.resolve(), (ROOT/'test'/cls/p.name))
    print('Data split created at', ROOT)
else:
    print('Using existing splits at', ROOT)

# Create generators
IMG_SIZE = (224,224)
BATCH = 32
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20, horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(ROOT/'train', target_size=IMG_SIZE, batch_size=BATCH, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(ROOT/'val', target_size=IMG_SIZE, batch_size=BATCH, class_mode='categorical')

num_classes = len(train_gen.class_indices)
print('Number of classes:', num_classes)


## Phase 2 — Model Development (Transfer Learning)

We build a MobileNetV2 transfer learning model as a baseline.

In [ ]:
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='avg')
for layer in base.layers:
    layer.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train for a few epochs (for demo; increase for real training)
EPOCHS = 5
history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS)
model.save('models/mobilenetv2_finetuned.h5')
print('Model saved to models/mobilenetv2_finetuned.h5')


### Evaluate on test set

In [ ]:
test_gen = val_datagen.flow_from_directory(ROOT/'test', target_size=IMG_SIZE, batch_size=BATCH, class_mode='categorical', shuffle=False)

preds = model.predict(test_gen, verbose=1)
y_true = test_gen.classes
y_pred = preds.argmax(axis=1)
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))

cm = confusion_matrix(y_true, y_pred)
print('Confusion matrix shape:', cm.shape)


## Phase 3 — Model Optimization (Ensemble with Classical ML)

Extract deep features using backbone and train a RandomForest classifier on them as an ensemble component.

In [ ]:
# Extract deep features from the backbone for each image in train/val/test
from tensorflow.keras.preprocessing import image

backbone = tf.keras.Model(inputs=base.input, outputs=base.output)

def extract_features_from_generator(gen, backbone_model):
    features = []
    labels = []
    gen.reset()
    for i in range(len(gen)):
        x, y = gen[i]
        feats = backbone_model.predict(x)
        features.append(feats)
        labels.append(y)
    X = np.vstack(features)
    y = np.vstack(labels).argmax(axis=1)
    return X, y

X_train, y_train = extract_features_from_generator(train_gen, backbone)
X_val, y_val = extract_features_from_generator(val_gen, backbone)

print('Feature shapes:', X_train.shape, X_val.shape)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print('RF trained. Validation score:', rf.score(X_val, y_val))

# Save RF with joblib
import joblib
os.makedirs('models', exist_ok=True)
joblib.dump(rf, 'models/rf_on_deep_features.pkl')
print('Saved RF to models/rf_on_deep_features.pkl')


## Phase 4 — Explainable AI (Grad-CAM)

Utility to generate Grad-CAM heatmap for Keras models.

In [ ]:
import cv2
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    # img_array: preprocessed image (1,H,W,3)
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap).numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (heatmap.max() + 1e-8)
    heatmap = cv2.resize(heatmap, (224,224))
    return heatmap

# Example on a test image
img_path = list((ROOT/'test').rglob('*.*'))[0]
img = image.load_img(img_path, target_size=(224,224))
img_arr = image.img_to_array(img)
img_proc = preprocess_input(img_arr.copy())
img_input = np.expand_dims(img_proc, 0)

# find last conv layer name
last_conv = None
for l in reversed(model.layers):
    if 'conv' in l.name:
        last_conv = l.name
        break
print('Last conv layer:', last_conv)
heatmap = make_gradcam_heatmap(img_input, model, last_conv)

# overlay
orig = (img_arr / 255.0)
heatmap_rgb = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
heatmap_rgb = cv2.cvtColor(heatmap_rgb, cv2.COLOR_BGR2RGB) / 255.0
overlay = 0.6*orig + 0.4*heatmap_rgb

plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.title('Original'); plt.axis('off'); plt.imshow(orig)
plt.subplot(1,2,2); plt.title('Grad-CAM'); plt.axis('off'); plt.imshow(overlay)
plt.show()


### Complementary feature — Recommendation (text-sim based)

In [ ]:
# Simple TF-IDF recommendation using sqlite DB created in the project
DB = 'data/medicinal_info.db'
conn = sqlite3.connect(DB)
# if db empty, create demo entries
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS plants (species TEXT PRIMARY KEY, medicinal_uses TEXT)')
rows = c.execute('SELECT COUNT(*) FROM plants').fetchone()[0]
if rows == 0:
    demo = [
        ('Azadirachta_indica','antibacterial; skin infection; wound healing'),
        ('Ocimum_sanctum','respiratory; immunity; anti-inflammatory'),
        ('Aloe_vera','wound healing; skin soothing; digestive')
    ]
    c.executemany('INSERT OR IGNORE INTO plants (species, medicinal_uses) VALUES (?,?)', demo)
    conn.commit()

species_list = [r[0] for r in c.execute('SELECT species FROM plants').fetchall()]
texts = [r[0] for r in c.execute('SELECT medicinal_uses FROM plants').fetchall()]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vec = TfidfVectorizer(stop_words='english')
X = vec.fit_transform(texts)

query_idx = 0
sims = cosine_similarity(X[query_idx], X)[0]
ranked = sorted(list(enumerate(sims)), key=lambda x:x[1], reverse=True)
print('Recommendations for', species_list[query_idx])
for idx,score in ranked[1:3]:
    print('-', species_list[idx], f'(score={score:.2f})')

conn.close()


## Phase 5 — Integration & Testing

Below is a minimal Streamlit app snippet you can use to integrate the model for demo.

In [ ]:
streamlit_code = '''
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import cv2

model = load_model('models/mobilenetv2_finetuned.h5')

st.title('Medicinal Plant Detector')
img_file = st.file_uploader('Upload leaf image', type=['jpg','png','jpeg'])
if img_file:
    arr = np.frombuffer(img_file.read(), np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    st.image(img, caption='Uploaded', use_column_width=True)
    x = cv2.resize(img, (224,224))
    x = preprocess_input(x.astype('float32'))
    pred = model.predict(np.expand_dims(x,0))[0]
    label = pred.argmax()
    st.write('Predicted class index:', int(label))
'''

with open('streamlit_demo.py','w') as f:
    f.write(streamlit_code)

print('Streamlit demo saved to streamlit_demo.py')


### Notebook Ready

You can download the notebook file from the workspace. Run cells step-by-step. For a production run, increase epochs and use GPU runtime.